In [1]:
## This file implements neural networks and random forest on p0006presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 62.39% for over-sampling data and 70.94%  after improvement.
## And it is 87.10% for combination data and 96.77% after improvement and 83.87% after regularization.
## For random forest, the accuracy is 69.23% for over-sampling data and 100% for combination data.
## For random forest with cross-validation, the mean accuracy is 69.63% for over-sampling, and 87.71% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0006presabsSTCC_qual.csv')
df.shape

(255, 88)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     0
15     0
16     1
17     1
18     2
19     0
20     1
21     1
22     0
23     0
24     2
25     0
26     2
27     0
28     0
29     2
      ..
225    1
226    1
227    1
228    2
229    2
230    1
231    0
232    1
233    1
234    1
235    2
236    0
237    0
238    0
239    0
240    0
241    1
242    0
243    0
244    1
245    0
246    1
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    130
1     86
2     39
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 87)

In [10]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 86) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 130), (1, 130), (2, 130)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 27), (1, 11), (2, 64)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/100
273/273 [==============================] - 0s 456us/step - loss: 20.2245 - accuracy: 0.3663 - val_loss: 25.2849 - val_accuracy: 0.4188
Epoch 2/100
273/273 [==============================] - 0s 96us/step - loss: 15.6775 - accuracy: 0.3810 - val_loss: 19.6003 - val_accuracy: 0.3846
Epoch 3/100
273/273 [==============================] - 0s 130us/step - loss: 12.0373 - accuracy: 0.3810 - val_loss: 13.4184 - val_accuracy: 0.4103
Epoch 4/100
273/273 [==============================] - 0s 107us/step - loss: 7.9275 - accuracy: 0.4139 - val_loss: 7.6572 - val_accuracy: 0.3675
Epoch 5/100
273/273 [==============================] - 0s 116us/step - loss: 4.3770 - accuracy: 0.4322 - val_loss: 4.7052 - val_accuracy: 0.3761
Epoch 6/100
273/273 [==============================] - 0s 114us/step - loss: 3.3455 - accuracy: 0.4066 - val_loss: 3.7051 - val_accuracy: 0.3761
Epoch 7/100
273/273 [==============================] - 0s 120us/step - loss: 2.

Epoch 57/100
273/273 [==============================] - ETA: 0s - loss: 0.7126 - accuracy: 0.71 - 0s 139us/step - loss: 0.6373 - accuracy: 0.7473 - val_loss: 0.8382 - val_accuracy: 0.6496
Epoch 58/100
273/273 [==============================] - 0s 191us/step - loss: 0.6411 - accuracy: 0.7509 - val_loss: 0.8729 - val_accuracy: 0.5983
Epoch 59/100
273/273 [==============================] - 0s 98us/step - loss: 0.6379 - accuracy: 0.7363 - val_loss: 0.8534 - val_accuracy: 0.6410
Epoch 60/100
273/273 [==============================] - 0s 116us/step - loss: 0.6318 - accuracy: 0.7436 - val_loss: 0.8611 - val_accuracy: 0.6410
Epoch 61/100
273/273 [==============================] - 0s 87us/step - loss: 0.6388 - accuracy: 0.7473 - val_loss: 0.9087 - val_accuracy: 0.6154
Epoch 62/100
273/273 [==============================] - 0s 82us/step - loss: 0.6955 - accuracy: 0.7289 - val_loss: 0.8605 - val_accuracy: 0.6239
Epoch 63/100
273/273 [==============================] - 0s 77us/step - loss: 0.6467 -

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 99us/step
over-sampling test accuracy: 62.39%


In [22]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/1000
273/273 [==============================] - 0s 949us/step - loss: 15.3978 - accuracy: 0.3297 - val_loss: 15.7429 - val_accuracy: 0.2479
Epoch 2/1000
273/273 [==============================] - 0s 89us/step - loss: 9.1625 - accuracy: 0.2821 - val_loss: 9.5237 - val_accuracy: 0.3590
Epoch 3/1000
273/273 [==============================] - 0s 148us/step - loss: 5.8545 - accuracy: 0.3223 - val_loss: 3.8735 - val_accuracy: 0.3333
Epoch 4/1000
273/273 [==============================] - 0s 139us/step - loss: 2.4723 - accuracy: 0.3333 - val_loss: 2.9694 - val_accuracy: 0.3504
Epoch 5/1000
273/273 [==============================] - 0s 180us/step - loss: 2.6528 - accuracy: 0.3736 - val_loss: 3.8306 - val_accuracy: 0.3248
Epoch 6/1000
273/273 [==============================] - 0s 175us/step - loss: 2.4798 - accuracy: 0.3883 - val_loss: 2.1760 - val_accuracy: 0.3846
Epoch 7/1000
273/273 [==============================] - 0s 123us/step - loss:

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.108430). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


273/273 [==============================] - 0s 345us/step - loss: 0.7906 - accuracy: 0.6703 - val_loss: 0.9135 - val_accuracy: 0.5983
Epoch 31/1000
273/273 [==============================] - 0s 334us/step - loss: 0.7909 - accuracy: 0.6703 - val_loss: 0.9039 - val_accuracy: 0.6325
Epoch 32/1000
273/273 [==============================] - 0s 129us/step - loss: 0.7934 - accuracy: 0.6593 - val_loss: 0.9639 - val_accuracy: 0.5726
Epoch 33/1000
273/273 [==============================] - 0s 98us/step - loss: 0.8021 - accuracy: 0.6557 - val_loss: 0.9174 - val_accuracy: 0.5470
Epoch 34/1000
273/273 [==============================] - 0s 115us/step - loss: 0.8355 - accuracy: 0.6520 - val_loss: 0.9663 - val_accuracy: 0.5556
Epoch 35/1000
273/273 [==============================] - 0s 124us/step - loss: 0.8177 - accuracy: 0.6740 - val_loss: 0.9779 - val_accuracy: 0.5385
Epoch 36/1000
273/273 [==============================] - 0s 116us/step - loss: 0.8133 - accuracy: 0.6557 - val_loss: 0.9184 - val_acc

273/273 [==============================] - 0s 56us/step - loss: 0.9459 - accuracy: 0.6667 - val_loss: 1.1731 - val_accuracy: 0.5641
Epoch 87/1000
273/273 [==============================] - 0s 62us/step - loss: 1.0717 - accuracy: 0.7143 - val_loss: 2.2269 - val_accuracy: 0.6154
Epoch 88/1000
273/273 [==============================] - 0s 66us/step - loss: 1.5789 - accuracy: 0.7106 - val_loss: 2.3515 - val_accuracy: 0.6154
Epoch 89/1000
273/273 [==============================] - 0s 55us/step - loss: 1.4818 - accuracy: 0.7253 - val_loss: 1.9263 - val_accuracy: 0.5983
Epoch 90/1000
273/273 [==============================] - 0s 59us/step - loss: 1.1082 - accuracy: 0.7289 - val_loss: 1.1380 - val_accuracy: 0.6068
Epoch 91/1000
273/273 [==============================] - 0s 58us/step - loss: 0.7099 - accuracy: 0.7289 - val_loss: 1.4274 - val_accuracy: 0.4615
Epoch 92/1000
273/273 [==============================] - 0s 58us/step - loss: 0.7485 - accuracy: 0.6996 - val_loss: 1.2150 - val_accuracy:

Epoch 142/1000
273/273 [==============================] - 0s 51us/step - loss: 0.6531 - accuracy: 0.7253 - val_loss: 1.0436 - val_accuracy: 0.5983
Epoch 143/1000
273/273 [==============================] - 0s 88us/step - loss: 0.6401 - accuracy: 0.7399 - val_loss: 0.9659 - val_accuracy: 0.5983
Epoch 144/1000
273/273 [==============================] - 0s 64us/step - loss: 0.5929 - accuracy: 0.7509 - val_loss: 0.9810 - val_accuracy: 0.5812
Epoch 145/1000
273/273 [==============================] - 0s 72us/step - loss: 0.5780 - accuracy: 0.7546 - val_loss: 1.0512 - val_accuracy: 0.5983
Epoch 146/1000
273/273 [==============================] - 0s 48us/step - loss: 0.5938 - accuracy: 0.7582 - val_loss: 0.9444 - val_accuracy: 0.6068
Epoch 147/1000
273/273 [==============================] - 0s 80us/step - loss: 0.5487 - accuracy: 0.7729 - val_loss: 0.9148 - val_accuracy: 0.6325
Epoch 148/1000
273/273 [==============================] - 0s 65us/step - loss: 0.5275 - accuracy: 0.7729 - val_loss: 0

...

Epoch 757/1000
273/273 [==============================] - 0s 91us/step - loss: 0.3322 - accuracy: 0.8535 - val_loss: 1.1322 - val_accuracy: 0.6923
Epoch 758/1000
273/273 [==============================] - 0s 38us/step - loss: 0.3391 - accuracy: 0.8425 - val_loss: 1.1197 - val_accuracy: 0.6838
Epoch 759/1000
273/273 [==============================] - 0s 64us/step - loss: 0.3389 - accuracy: 0.8498 - val_loss: 1.1203 - val_accuracy: 0.6752
Epoch 760/1000
273/273 [==============================] - 0s 64us/step - loss: 0.3345 - accuracy: 0.8498 - val_loss: 1.1037 - val_accuracy: 0.7009
Epoch 761/1000
273/273 [==============================] - 0s 69us/step - loss: 0.3453 - accuracy: 0.8315 - val_loss: 1.1553 - val_accuracy: 0.6923
Epoch 762/1000
273/273 [==============================] - 0s 38us/step - loss: 0.3977 - accuracy: 0.8388 - val_loss: 1.1116 - val_accuracy: 0.7009
Epoch 763/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3631 - accuracy: 0.8352 - val_loss: 1

Epoch 813/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3645 - accuracy: 0.8278 - val_loss: 1.1499 - val_accuracy: 0.6667
Epoch 814/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3604 - accuracy: 0.8425 - val_loss: 1.1344 - val_accuracy: 0.6923
Epoch 815/1000
273/273 [==============================] - 0s 80us/step - loss: 0.3407 - accuracy: 0.8425 - val_loss: 1.1444 - val_accuracy: 0.6838
Epoch 816/1000
273/273 [==============================] - 0s 65us/step - loss: 0.3460 - accuracy: 0.8462 - val_loss: 1.1287 - val_accuracy: 0.6581
Epoch 817/1000
273/273 [==============================] - 0s 86us/step - loss: 0.3336 - accuracy: 0.8388 - val_loss: 1.1419 - val_accuracy: 0.6581
Epoch 818/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3286 - accuracy: 0.8462 - val_loss: 1.1533 - val_accuracy: 0.6667
Epoch 819/1000
273/273 [==============================] - 0s 104us/step - loss: 0.3375 - accuracy: 0.8352 - val_loss: 

Epoch 869/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3425 - accuracy: 0.8425 - val_loss: 1.1960 - val_accuracy: 0.6752
Epoch 870/1000
273/273 [==============================] - 0s 64us/step - loss: 0.3413 - accuracy: 0.8462 - val_loss: 1.2002 - val_accuracy: 0.6752
Epoch 871/1000
273/273 [==============================] - 0s 61us/step - loss: 0.3360 - accuracy: 0.8388 - val_loss: 1.1756 - val_accuracy: 0.6923
Epoch 872/1000
273/273 [==============================] - 0s 60us/step - loss: 0.3293 - accuracy: 0.8498 - val_loss: 1.1514 - val_accuracy: 0.7009
Epoch 873/1000
273/273 [==============================] - 0s 60us/step - loss: 0.3358 - accuracy: 0.8498 - val_loss: 1.1419 - val_accuracy: 0.7094
Epoch 874/1000
273/273 [==============================] - 0s 59us/step - loss: 0.3281 - accuracy: 0.8425 - val_loss: 1.1417 - val_accuracy: 0.7094
Epoch 875/1000
273/273 [==============================] - 0s 58us/step - loss: 0.3233 - accuracy: 0.8498 - val_loss: 1

Epoch 925/1000
273/273 [==============================] - 0s 67us/step - loss: 0.3430 - accuracy: 0.8571 - val_loss: 1.2103 - val_accuracy: 0.6667
Epoch 926/1000
273/273 [==============================] - 0s 61us/step - loss: 0.3254 - accuracy: 0.8462 - val_loss: 1.2243 - val_accuracy: 0.6838
Epoch 927/1000
273/273 [==============================] - 0s 74us/step - loss: 0.3661 - accuracy: 0.8498 - val_loss: 1.2172 - val_accuracy: 0.6838
Epoch 928/1000
273/273 [==============================] - 0s 61us/step - loss: 0.3411 - accuracy: 0.8462 - val_loss: 1.2168 - val_accuracy: 0.6838
Epoch 929/1000
273/273 [==============================] - 0s 61us/step - loss: 0.3440 - accuracy: 0.8462 - val_loss: 1.2225 - val_accuracy: 0.6838
Epoch 930/1000
273/273 [==============================] - 0s 66us/step - loss: 0.3419 - accuracy: 0.8645 - val_loss: 1.1995 - val_accuracy: 0.6752
Epoch 931/1000
273/273 [==============================] - 0s 110us/step - loss: 0.3334 - accuracy: 0.8498 - val_loss: 

273/273 [==============================] - 0s 92us/step - loss: 0.6652 - accuracy: 0.7766 - val_loss: 1.8297 - val_accuracy: 0.6496
Epoch 981/1000
273/273 [==============================] - 0s 80us/step - loss: 0.5287 - accuracy: 0.8242 - val_loss: 1.3768 - val_accuracy: 0.6838
Epoch 982/1000
273/273 [==============================] - 0s 66us/step - loss: 0.5618 - accuracy: 0.8278 - val_loss: 1.4988 - val_accuracy: 0.6667
Epoch 983/1000
273/273 [==============================] - 0s 93us/step - loss: 0.3929 - accuracy: 0.8388 - val_loss: 1.2315 - val_accuracy: 0.6667
Epoch 984/1000
273/273 [==============================] - 0s 67us/step - loss: 0.3375 - accuracy: 0.8498 - val_loss: 1.2597 - val_accuracy: 0.6923
Epoch 985/1000
273/273 [==============================] - 0s 67us/step - loss: 0.4100 - accuracy: 0.8535 - val_loss: 1.1997 - val_accuracy: 0.6923
Epoch 986/1000
273/273 [==============================] - 0s 66us/step - loss: 0.3527 - accuracy: 0.8425 - val_loss: 1.2181 - val_acc

In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 42us/step
over-sampling test accuracy: 70.94%


In [45]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [46]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/100
71/71 [==============================] - 0s 2ms/step - loss: 17.1772 - accuracy: 0.2394 - val_loss: 13.4167 - val_accuracy: 0.2903
Epoch 2/100
71/71 [==============================] - 0s 139us/step - loss: 14.7014 - accuracy: 0.2394 - val_loss: 11.1196 - val_accuracy: 0.2903
Epoch 3/100
71/71 [==============================] - 0s 145us/step - loss: 12.5473 - accuracy: 0.2394 - val_loss: 8.8545 - val_accuracy: 0.2903
Epoch 4/100
71/71 [==============================] - 0s 214us/step - loss: 10.4743 - accuracy: 0.3380 - val_loss: 6.6807 - val_accuracy: 0.4516
Epoch 5/100
71/71 [==============================] - 0s 273us/step - loss: 8.3638 - accuracy: 0.3521 - val_loss: 5.0251 - val_accuracy: 0.4516
Epoch 6/100
71/71 [==============================] - 0s 162us/step - loss: 7.1776 - accuracy: 0.3380 - val_loss: 4.8133 - val_accuracy: 0.5806
Epoch 7/100
71/71 [==============================] - 0s 321us/step - loss: 6.4541 - accuracy: 

71/71 [==============================] - 0s 221us/step - loss: 0.5561 - accuracy: 0.8451 - val_loss: 0.7173 - val_accuracy: 0.8065
Epoch 58/100
71/71 [==============================] - 0s 215us/step - loss: 0.7490 - accuracy: 0.8451 - val_loss: 0.7203 - val_accuracy: 0.8065
Epoch 59/100
71/71 [==============================] - 0s 294us/step - loss: 0.7267 - accuracy: 0.8310 - val_loss: 0.6093 - val_accuracy: 0.8065
Epoch 60/100
71/71 [==============================] - 0s 220us/step - loss: 0.5828 - accuracy: 0.8451 - val_loss: 0.5134 - val_accuracy: 0.7742
Epoch 61/100
71/71 [==============================] - 0s 299us/step - loss: 0.4747 - accuracy: 0.8310 - val_loss: 0.5803 - val_accuracy: 0.7419
Epoch 62/100
71/71 [==============================] - 0s 207us/step - loss: 0.4746 - accuracy: 0.8310 - val_loss: 0.5691 - val_accuracy: 0.8065
Epoch 63/100
71/71 [==============================] - 0s 161us/step - loss: 0.4940 - accuracy: 0.8310 - val_loss: 0.7137 - val_accuracy: 0.7419
Epoch

In [48]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

31/31 [==============================] - 0s 106us/step
combination test accuracy: 87.10%


In [64]:
#### improve neural network on combination data
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [65]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/1000
71/71 [==============================] - 0s 2ms/step - loss: 26.0420 - accuracy: 0.1127 - val_loss: 26.4464 - val_accuracy: 0.0968
Epoch 2/1000
71/71 [==============================] - 0s 135us/step - loss: 20.2613 - accuracy: 0.1127 - val_loss: 21.4129 - val_accuracy: 0.0968
Epoch 3/1000
71/71 [==============================] - 0s 146us/step - loss: 15.5129 - accuracy: 0.1127 - val_loss: 16.6066 - val_accuracy: 0.0968
Epoch 4/1000
71/71 [==============================] - 0s 167us/step - loss: 11.1381 - accuracy: 0.1127 - val_loss: 13.7302 - val_accuracy: 0.1290
Epoch 5/1000
71/71 [==============================] - 0s 184us/step - loss: 9.0404 - accuracy: 0.1831 - val_loss: 12.9512 - val_accuracy: 0.1935
Epoch 6/1000
71/71 [==============================] - 0s 216us/step - loss: 8.7317 - accuracy: 0.2113 - val_loss: 11.9218 - val_accuracy: 0.2258
Epoch 7/1000
71/71 [==============================] - 0s 186us/step - loss: 8.1388 -

Epoch 57/1000
71/71 [==============================] - 0s 211us/step - loss: 1.8784 - accuracy: 0.8592 - val_loss: 1.5510 - val_accuracy: 0.7419
Epoch 58/1000
71/71 [==============================] - 0s 239us/step - loss: 1.8402 - accuracy: 0.8169 - val_loss: 1.3294 - val_accuracy: 0.7419
Epoch 59/1000
71/71 [==============================] - 0s 143us/step - loss: 1.6928 - accuracy: 0.8169 - val_loss: 1.1841 - val_accuracy: 0.8387
Epoch 60/1000
71/71 [==============================] - 0s 156us/step - loss: 1.3883 - accuracy: 0.8310 - val_loss: 0.8760 - val_accuracy: 0.8065
Epoch 61/1000
71/71 [==============================] - 0s 191us/step - loss: 0.8861 - accuracy: 0.8169 - val_loss: 0.5593 - val_accuracy: 0.7742
Epoch 62/1000
71/71 [==============================] - 0s 151us/step - loss: 0.5075 - accuracy: 0.8310 - val_loss: 1.1999 - val_accuracy: 0.7419
Epoch 63/1000
71/71 [==============================] - 0s 176us/step - loss: 1.1572 - accuracy: 0.8169 - val_loss: 1.2518 - val_ac

71/71 [==============================] - 0s 187us/step - loss: 0.6246 - accuracy: 0.8592 - val_loss: 0.6074 - val_accuracy: 0.8387
Epoch 114/1000
71/71 [==============================] - 0s 153us/step - loss: 0.5986 - accuracy: 0.8451 - val_loss: 0.3466 - val_accuracy: 0.8065
Epoch 115/1000
71/71 [==============================] - 0s 150us/step - loss: 0.2981 - accuracy: 0.8592 - val_loss: 0.3357 - val_accuracy: 0.8065
Epoch 116/1000
71/71 [==============================] - 0s 176us/step - loss: 0.2928 - accuracy: 0.8732 - val_loss: 0.2946 - val_accuracy: 0.8710
Epoch 117/1000
71/71 [==============================] - 0s 260us/step - loss: 0.2646 - accuracy: 0.8592 - val_loss: 0.3234 - val_accuracy: 0.8387
Epoch 118/1000
71/71 [==============================] - 0s 150us/step - loss: 0.2514 - accuracy: 0.8873 - val_loss: 0.3169 - val_accuracy: 0.8387
Epoch 119/1000
71/71 [==============================] - 0s 140us/step - loss: 0.2617 - accuracy: 0.8873 - val_loss: 0.3152 - val_accuracy: 

71/71 [==============================] - 0s 157us/step - loss: 0.3420 - accuracy: 0.9155 - val_loss: 0.3713 - val_accuracy: 0.9032
Epoch 170/1000
71/71 [==============================] - 0s 218us/step - loss: 0.2735 - accuracy: 0.9296 - val_loss: 0.5406 - val_accuracy: 0.8710
Epoch 171/1000
71/71 [==============================] - 0s 177us/step - loss: 0.2805 - accuracy: 0.9155 - val_loss: 0.1991 - val_accuracy: 0.9355
Epoch 172/1000
71/71 [==============================] - 0s 142us/step - loss: 0.2441 - accuracy: 0.9296 - val_loss: 0.3135 - val_accuracy: 0.9677
Epoch 173/1000
71/71 [==============================] - 0s 226us/step - loss: 0.4637 - accuracy: 0.9155 - val_loss: 0.2984 - val_accuracy: 0.9355
Epoch 174/1000
71/71 [==============================] - 0s 217us/step - loss: 0.4090 - accuracy: 0.9155 - val_loss: 0.2983 - val_accuracy: 0.9032
Epoch 175/1000
71/71 [==============================] - 0s 162us/step - loss: 0.2745 - accuracy: 0.9014 - val_loss: 0.2095 - val_accuracy: 

71/71 [==============================] - 0s 173us/step - loss: 0.2654 - accuracy: 0.9437 - val_loss: 0.2337 - val_accuracy: 0.9355
Epoch 226/1000
71/71 [==============================] - 0s 155us/step - loss: 0.1586 - accuracy: 0.9296 - val_loss: 0.2297 - val_accuracy: 0.9032
Epoch 227/1000
71/71 [==============================] - 0s 161us/step - loss: 0.1942 - accuracy: 0.9296 - val_loss: 0.1369 - val_accuracy: 0.9355
Epoch 228/1000
71/71 [==============================] - 0s 165us/step - loss: 0.1581 - accuracy: 0.9296 - val_loss: 0.2683 - val_accuracy: 0.9355
Epoch 229/1000
71/71 [==============================] - 0s 171us/step - loss: 0.2406 - accuracy: 0.9155 - val_loss: 0.1351 - val_accuracy: 0.9677
Epoch 230/1000
71/71 [==============================] - 0s 216us/step - loss: 0.0992 - accuracy: 0.9437 - val_loss: 0.5002 - val_accuracy: 0.9032
Epoch 231/1000
71/71 [==============================] - 0s 193us/step - loss: 0.2906 - accuracy: 0.9296 - val_loss: 0.6536 - val_accuracy: 

...

71/71 [==============================] - 0s 204us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.0633 - val_accuracy: 0.9677
Epoch 730/1000
71/71 [==============================] - 0s 148us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.0618 - val_accuracy: 0.9677
Epoch 731/1000
71/71 [==============================] - 0s 214us/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0614 - val_accuracy: 0.9677
Epoch 732/1000
71/71 [==============================] - 0s 131us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.0620 - val_accuracy: 0.9677
Epoch 733/1000
71/71 [==============================] - 0s 147us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0631 - val_accuracy: 0.9677
Epoch 734/1000
71/71 [==============================] - 0s 240us/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.0611 - val_accuracy: 0.9677
Epoch 735/1000
71/71 [==============================] - 0s 139us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.0587 - val_accuracy: 

71/71 [==============================] - 0s 158us/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.2699 - val_accuracy: 0.9355
Epoch 786/1000
71/71 [==============================] - 0s 249us/step - loss: 0.2401 - accuracy: 0.9577 - val_loss: 0.0700 - val_accuracy: 0.9677
Epoch 787/1000
71/71 [==============================] - 0s 209us/step - loss: 0.0802 - accuracy: 0.9718 - val_loss: 0.1466 - val_accuracy: 0.9355
Epoch 788/1000
71/71 [==============================] - 0s 229us/step - loss: 0.1679 - accuracy: 0.9577 - val_loss: 0.1112 - val_accuracy: 0.9355
Epoch 789/1000
71/71 [==============================] - 0s 249us/step - loss: 0.0922 - accuracy: 0.9718 - val_loss: 0.0827 - val_accuracy: 0.9677
Epoch 790/1000
71/71 [==============================] - 0s 231us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.1234 - val_accuracy: 0.9355
Epoch 791/1000
71/71 [==============================] - 0s 167us/step - loss: 0.0422 - accuracy: 0.9718 - val_loss: 0.0841 - val_accuracy: 

71/71 [==============================] - 0s 190us/step - loss: 0.0111 - accuracy: 1.0000 - val_loss: 0.0755 - val_accuracy: 0.9677
Epoch 842/1000
71/71 [==============================] - 0s 189us/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0718 - val_accuracy: 0.9677
Epoch 843/1000
71/71 [==============================] - 0s 183us/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9677
Epoch 844/1000
71/71 [==============================] - 0s 168us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.0707 - val_accuracy: 0.9677
Epoch 845/1000
71/71 [==============================] - 0s 133us/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0717 - val_accuracy: 0.9677
Epoch 846/1000
71/71 [==============================] - 0s 196us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0730 - val_accuracy: 0.9677
Epoch 847/1000
71/71 [==============================] - 0s 138us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0740 - val_accuracy: 

71/71 [==============================] - 0s 158us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0707 - val_accuracy: 0.9677
Epoch 898/1000
71/71 [==============================] - 0s 153us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9677
Epoch 899/1000
71/71 [==============================] - 0s 150us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9677
Epoch 900/1000
71/71 [==============================] - 0s 133us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0705 - val_accuracy: 0.9677
Epoch 901/1000
71/71 [==============================] - 0s 180us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0693 - val_accuracy: 0.9677
Epoch 902/1000
71/71 [==============================] - 0s 144us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0686 - val_accuracy: 0.9677
Epoch 903/1000
71/71 [==============================] - 0s 197us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0691 - val_accuracy: 

71/71 [==============================] - 0s 239us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.9677
Epoch 954/1000
71/71 [==============================] - 0s 167us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9677
Epoch 955/1000
71/71 [==============================] - 0s 159us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0708 - val_accuracy: 0.9677
Epoch 956/1000
71/71 [==============================] - 0s 149us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0715 - val_accuracy: 0.9677
Epoch 957/1000
71/71 [==============================] - 0s 168us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0708 - val_accuracy: 0.9677
Epoch 958/1000
71/71 [==============================] - 0s 137us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0693 - val_accuracy: 0.9677
Epoch 959/1000
71/71 [==============================] - 0s 165us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0676 - val_accuracy: 

In [67]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

31/31 [==============================] - 0s 95us/step
combination test accuracy: 96.77%


In [60]:
#### add dropout
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [61]:
model1_comb3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [62]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/1000
71/71 [==============================] - 0s 3ms/step - loss: 22.1567 - accuracy: 0.2817 - val_loss: 20.2457 - val_accuracy: 0.2903
Epoch 2/1000
71/71 [==============================] - 0s 140us/step - loss: 21.5886 - accuracy: 0.2676 - val_loss: 17.7606 - val_accuracy: 0.4194
Epoch 3/1000
71/71 [==============================] - 0s 194us/step - loss: 21.5658 - accuracy: 0.4085 - val_loss: 17.6955 - val_accuracy: 0.7097
Epoch 4/1000
71/71 [==============================] - 0s 202us/step - loss: 18.2624 - accuracy: 0.6056 - val_loss: 17.4848 - val_accuracy: 0.7097
Epoch 5/1000
71/71 [==============================] - 0s 301us/step - loss: 20.6467 - accuracy: 0.5915 - val_loss: 17.2421 - val_accuracy: 0.7419
Epoch 6/1000
71/71 [==============================] - 0s 215us/step - loss: 20.4037 - accuracy: 0.6620 - val_loss: 16.9475 - val_accuracy: 0.7419
Epoch 7/1000
71/71 [==============================] - 0s 222us/step - loss: 21.123

Epoch 57/1000
71/71 [==============================] - 0s 391us/step - loss: 7.1981 - accuracy: 0.6620 - val_loss: 5.0131 - val_accuracy: 0.7419
Epoch 58/1000
71/71 [==============================] - 0s 184us/step - loss: 7.3631 - accuracy: 0.6479 - val_loss: 4.8800 - val_accuracy: 0.7419
Epoch 59/1000
71/71 [==============================] - 0s 195us/step - loss: 6.7549 - accuracy: 0.6901 - val_loss: 4.7421 - val_accuracy: 0.7419
Epoch 60/1000
71/71 [==============================] - 0s 318us/step - loss: 6.3295 - accuracy: 0.6479 - val_loss: 4.6109 - val_accuracy: 0.7419
Epoch 61/1000
71/71 [==============================] - 0s 193us/step - loss: 6.5400 - accuracy: 0.5775 - val_loss: 4.4932 - val_accuracy: 0.7742
Epoch 62/1000
71/71 [==============================] - 0s 202us/step - loss: 6.5684 - accuracy: 0.6197 - val_loss: 4.3741 - val_accuracy: 0.7742
Epoch 63/1000
71/71 [==============================] - 0s 160us/step - loss: 6.3071 - accuracy: 0.6479 - val_loss: 4.2377 - val_ac

71/71 [==============================] - 0s 1ms/step - loss: 2.8233 - accuracy: 0.6056 - val_loss: 1.0300 - val_accuracy: 0.7742
Epoch 114/1000
71/71 [==============================] - 0s 245us/step - loss: 1.8673 - accuracy: 0.6901 - val_loss: 1.0314 - val_accuracy: 0.7742
Epoch 115/1000
71/71 [==============================] - 0s 239us/step - loss: 2.3490 - accuracy: 0.6479 - val_loss: 1.0198 - val_accuracy: 0.7742
Epoch 116/1000
71/71 [==============================] - 0s 200us/step - loss: 3.0076 - accuracy: 0.6197 - val_loss: 1.0242 - val_accuracy: 0.7742
Epoch 117/1000
71/71 [==============================] - 0s 224us/step - loss: 2.6757 - accuracy: 0.6479 - val_loss: 1.0300 - val_accuracy: 0.7742
Epoch 118/1000
71/71 [==============================] - 0s 271us/step - loss: 1.9815 - accuracy: 0.6479 - val_loss: 1.0205 - val_accuracy: 0.7742
Epoch 119/1000
71/71 [==============================] - 0s 282us/step - loss: 2.6332 - accuracy: 0.6479 - val_loss: 1.0134 - val_accuracy: 0.

Epoch 169/1000
71/71 [==============================] - 0s 323us/step - loss: 1.7877 - accuracy: 0.7183 - val_loss: 0.5955 - val_accuracy: 0.8710
Epoch 170/1000
71/71 [==============================] - 0s 350us/step - loss: 1.6500 - accuracy: 0.6620 - val_loss: 0.5864 - val_accuracy: 0.8387
Epoch 171/1000
71/71 [==============================] - 0s 312us/step - loss: 1.8457 - accuracy: 0.7183 - val_loss: 0.5871 - val_accuracy: 0.8710
Epoch 172/1000
71/71 [==============================] - 0s 190us/step - loss: 1.7583 - accuracy: 0.7183 - val_loss: 0.5817 - val_accuracy: 0.8710
Epoch 173/1000
71/71 [==============================] - 0s 310us/step - loss: 1.4481 - accuracy: 0.7606 - val_loss: 0.5654 - val_accuracy: 0.8387
Epoch 174/1000
71/71 [==============================] - 0s 263us/step - loss: 1.8696 - accuracy: 0.6901 - val_loss: 0.5577 - val_accuracy: 0.8710
Epoch 175/1000
71/71 [==============================] - 0s 285us/step - loss: 1.5596 - accuracy: 0.6620 - val_loss: 0.5535 -

71/71 [==============================] - 0s 271us/step - loss: 1.7749 - accuracy: 0.7606 - val_loss: 1.2637 - val_accuracy: 0.8387
Epoch 226/1000
71/71 [==============================] - 0s 322us/step - loss: 2.3079 - accuracy: 0.6056 - val_loss: 1.0133 - val_accuracy: 0.8387
Epoch 227/1000
71/71 [==============================] - 0s 421us/step - loss: 1.8025 - accuracy: 0.7042 - val_loss: 0.4620 - val_accuracy: 0.9355
Epoch 228/1000
71/71 [==============================] - 0s 182us/step - loss: 1.5979 - accuracy: 0.7324 - val_loss: 0.4467 - val_accuracy: 0.9355
Epoch 229/1000
71/71 [==============================] - 0s 137us/step - loss: 1.6708 - accuracy: 0.6761 - val_loss: 0.4436 - val_accuracy: 0.9355
Epoch 230/1000
71/71 [==============================] - 0s 173us/step - loss: 1.7617 - accuracy: 0.6620 - val_loss: 0.4348 - val_accuracy: 0.9355
Epoch 231/1000
71/71 [==============================] - 0s 321us/step - loss: 1.9892 - accuracy: 0.7324 - val_loss: 0.4239 - val_accuracy: 

...

71/71 [==============================] - 0s 172us/step - loss: 1.1518 - accuracy: 0.7606 - val_loss: 0.2099 - val_accuracy: 0.9677
Epoch 730/1000
71/71 [==============================] - 0s 157us/step - loss: 0.9028 - accuracy: 0.8732 - val_loss: 0.2088 - val_accuracy: 0.9677
Epoch 731/1000
71/71 [==============================] - 0s 169us/step - loss: 0.8148 - accuracy: 0.7465 - val_loss: 0.2080 - val_accuracy: 0.9677
Epoch 732/1000
71/71 [==============================] - 0s 196us/step - loss: 0.9830 - accuracy: 0.7324 - val_loss: 0.2094 - val_accuracy: 0.9677
Epoch 733/1000
71/71 [==============================] - 0s 176us/step - loss: 0.9277 - accuracy: 0.7887 - val_loss: 0.2066 - val_accuracy: 0.9677
Epoch 734/1000
71/71 [==============================] - 0s 234us/step - loss: 1.2749 - accuracy: 0.7606 - val_loss: 0.5254 - val_accuracy: 0.8710
Epoch 735/1000
71/71 [==============================] - 0s 435us/step - loss: 1.3959 - accuracy: 0.6761 - val_loss: 0.6412 - val_accuracy: 

71/71 [==============================] - 0s 230us/step - loss: 1.5005 - accuracy: 0.7324 - val_loss: 0.2590 - val_accuracy: 0.9677
Epoch 786/1000
71/71 [==============================] - 0s 187us/step - loss: 1.2955 - accuracy: 0.7324 - val_loss: 0.3924 - val_accuracy: 0.9677
Epoch 787/1000
71/71 [==============================] - 0s 205us/step - loss: 0.7569 - accuracy: 0.8028 - val_loss: 0.3498 - val_accuracy: 0.9032
Epoch 788/1000
71/71 [==============================] - 0s 271us/step - loss: 1.3041 - accuracy: 0.6761 - val_loss: 0.4117 - val_accuracy: 0.9032
Epoch 789/1000
71/71 [==============================] - 0s 392us/step - loss: 1.1436 - accuracy: 0.7042 - val_loss: 0.4128 - val_accuracy: 0.9032
Epoch 790/1000
71/71 [==============================] - 0s 578us/step - loss: 1.8574 - accuracy: 0.6761 - val_loss: 0.4741 - val_accuracy: 0.8710
Epoch 791/1000
71/71 [==============================] - 0s 449us/step - loss: 0.8328 - accuracy: 0.7746 - val_loss: 0.5116 - val_accuracy: 

Epoch 841/1000
71/71 [==============================] - 0s 327us/step - loss: 0.9567 - accuracy: 0.7183 - val_loss: 0.5218 - val_accuracy: 0.8387
Epoch 842/1000
71/71 [==============================] - 0s 324us/step - loss: 1.4615 - accuracy: 0.7183 - val_loss: 0.5174 - val_accuracy: 0.8387
Epoch 843/1000
71/71 [==============================] - 0s 273us/step - loss: 0.7419 - accuracy: 0.7465 - val_loss: 0.5120 - val_accuracy: 0.8387
Epoch 844/1000
71/71 [==============================] - 0s 177us/step - loss: 0.7691 - accuracy: 0.7183 - val_loss: 0.5078 - val_accuracy: 0.8387
Epoch 845/1000
71/71 [==============================] - 0s 188us/step - loss: 0.9845 - accuracy: 0.6901 - val_loss: 0.5021 - val_accuracy: 0.8387
Epoch 846/1000
71/71 [==============================] - 0s 179us/step - loss: 1.6098 - accuracy: 0.6056 - val_loss: 0.4968 - val_accuracy: 0.8387
Epoch 847/1000
71/71 [==============================] - 0s 328us/step - loss: 1.1421 - accuracy: 0.7042 - val_loss: 0.4921 -

71/71 [==============================] - 0s 205us/step - loss: 0.7378 - accuracy: 0.6901 - val_loss: 0.4170 - val_accuracy: 0.8387
Epoch 898/1000
71/71 [==============================] - 0s 217us/step - loss: 0.7410 - accuracy: 0.6901 - val_loss: 0.4162 - val_accuracy: 0.8387
Epoch 899/1000
71/71 [==============================] - 0s 211us/step - loss: 1.5853 - accuracy: 0.6338 - val_loss: 0.4147 - val_accuracy: 0.8387
Epoch 900/1000
71/71 [==============================] - 0s 223us/step - loss: 1.2512 - accuracy: 0.7606 - val_loss: 0.4125 - val_accuracy: 0.8387
Epoch 901/1000
71/71 [==============================] - 0s 161us/step - loss: 1.3262 - accuracy: 0.7183 - val_loss: 0.4103 - val_accuracy: 0.8387
Epoch 902/1000
71/71 [==============================] - 0s 155us/step - loss: 0.8761 - accuracy: 0.7465 - val_loss: 0.4089 - val_accuracy: 0.8387
Epoch 903/1000
71/71 [==============================] - 0s 214us/step - loss: 1.3080 - accuracy: 0.7042 - val_loss: 0.4086 - val_accuracy: 

71/71 [==============================] - 0s 175us/step - loss: 1.3431 - accuracy: 0.6901 - val_loss: 0.4050 - val_accuracy: 0.8387
Epoch 954/1000
71/71 [==============================] - 0s 213us/step - loss: 1.4288 - accuracy: 0.7606 - val_loss: 0.4054 - val_accuracy: 0.8387
Epoch 955/1000
71/71 [==============================] - 0s 173us/step - loss: 1.0512 - accuracy: 0.7606 - val_loss: 0.4049 - val_accuracy: 0.8387
Epoch 956/1000
71/71 [==============================] - 0s 228us/step - loss: 1.2982 - accuracy: 0.7183 - val_loss: 0.4043 - val_accuracy: 0.8387
Epoch 957/1000
71/71 [==============================] - 0s 147us/step - loss: 1.3852 - accuracy: 0.6197 - val_loss: 0.4035 - val_accuracy: 0.8387
Epoch 958/1000
71/71 [==============================] - 0s 175us/step - loss: 1.5187 - accuracy: 0.6901 - val_loss: 0.4021 - val_accuracy: 0.8387
Epoch 959/1000
71/71 [==============================] - 0s 212us/step - loss: 0.7727 - accuracy: 0.8592 - val_loss: 0.4011 - val_accuracy: 

In [63]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

31/31 [==============================] - 0s 137us/step
combination test accuracy: 83.87%


In [38]:
############## Random Forest ##############

In [70]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state = 123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [71]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 69.23%


In [72]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state = 123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [73]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [68]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.66666667 0.68518519 0.68518519 0.7037037  0.74074074]
0.6962962962962964


In [69]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.93333333 0.86666667 0.8        0.78571429 1.        ]
0.8771428571428572
